In [4]:
!date

Tue Jul 11 11:28:55 EDT 2023


In [5]:
pwd

'/mmfs1/data/aglinska/BC-actionpred-seg'

In [6]:
#!pip install scikit-video

In [7]:
#!pip install opencv-python

In [8]:
from matplotlib import pyplot as plt
from PIL import Image, ImageEnhance,ImageStat
import skvideo
import skvideo.io
import numpy as np
import os
from tqdm import tqdm
import cv2

from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import pandas as pd
import random

In [9]:
fnt_dir = './Arial Bold.ttf'
fnt = ImageFont.truetype(fnt_dir, 40)

In [10]:
def load_video(fn):
    #fn = 'ID4_h1_1.mp4'
    videodata = skvideo.io.vread(fn)
    return videodata

In [11]:
def get_text_im(text='000',nframes=150):
    img = Image.new('RGB', (640,360))
    d = ImageDraw.Draw(img)
    d.text((320-len(text)*10, 150), text, fill=(255, 255, 255),font=fnt)
    return np.array([np.array(img) for _ in range(nframes)])

In [12]:
def save_video(videodata,ofn='outvideo.mp4'):
    #skvideo.io.vwrite(ofn, videodata.astype(np.uint8),outputdict={'-pix_fmt' : 'yuv420p','-codec': 'h264', '-filter:v' : 'fps=20'})
    skvideo.io.vwrite(ofn, videodata.astype(np.uint8))

In [13]:
def safe_mkdir(fn):
    import os
    if not os.path.exists(fn):
        os.mkdir(fn)

In [14]:
df = pd.read_csv('./Results/mTurk_masterfile.csv')
df.head(5)

,Unnamed: 0,action,cat_idx,file,code,label,choice_idx_1,choice_idx_2,choice_idx_3,choice_idx_4,choice_idx_5,choice_label_1,choice_label_2,choice_label_3,choice_label_4,choice_label_5
0,0,golf_part,24,NaN,NaN,golf part,24,40,20,34,4,golf part,play cymbals,eat apple,jack up car,balloon animal
1,1,using_lawn_mower,97,NaN,NaN,using lawn mower,97,32,23,69,40,using lawn mower,handshake dog,floss dance,pull ups,play cymbals
2,2,using_scythe,98,NaN,NaN,using scythe,98,23,31,103,113,using scythe,floss dance,handsaw,wear face mask,yoga firefly
3,3,closing_door,14,NaN,NaN,closing door,14,58,84,5,34,closing door,play serpent,softball pitch,basketball jabstep,jack up car
4,4,chainsaw_tree,10,NaN,NaN,chainsaw tree,10,18,19,60,17,chainsaw tree,dog highfive,draw handgun,play tambourine,dips


In [15]:
plt.ioff()

In [16]:
# indir_root = './Data/pytorch-data/xl_121_15_action_data_bg/test/'
# vid_fn_temp = './Data/haa500_v1_1_yolo_seg4/{this_cat}/{this_cat}_{vid_num}_item_0_inv_seg.mp4'
# ofdir = './Data/01_gifs_for_mTurk_bg/'
# safe_mkdir(ofdir)

In [17]:
#vid_fn_temp = './Data/haa500_v1_1_yolo_seg4/{this_cat}/{this_cat}_{vid_num}.mp4'
#vid_fn_temp = './Data/haa500_v1_1_yolo_seg4/{this_cat}/{this_cat}_{vid_num}_item_0_inp_seg.mp4'

In [18]:
mturk_categories = df['action'].values
n = len(mturk_categories)
mturk_categories_nice = df['label'].values

In [31]:
### ORIGINAL VIDEOS
indir_root = './Data/pytorch-data/xl_121_15_action_data_orig/test/'
vid_fn_temp = './Data/haa500_v1_1_yolo_seg4/{this_cat}/{this_cat}_{vid_num}.mp4'
ofdir = './Data/01_gifs_for_mTurk_orig/'
safe_mkdir(ofdir)

codes = [f'{np.random.randint(low=0,high=999):03d}' for _ in range(n)]
df['gif_code_orig'] = codes

for c in tqdm(range(n)):
    this_cat = mturk_categories[c]
    indir = os.path.join(indir_root,this_cat)
    frames = [frame for frame in os.listdir(indir) if not frame.startswith('.')]
    frames.sort()
    frame = frames[0]
    vid_num = frame.split('_')[0]
    vid_fn = vid_fn_temp.format(this_cat=this_cat,vid_num=vid_num)
    video_data = load_video(vid_fn)
    video_data = video_data[0:min(video_data.shape[0],150),:,:,:]
    
    arr = np.concatenate((
                get_text_im(text='5',nframes=25),
                get_text_im(text='4',nframes=25),
                get_text_im(text='3',nframes=25),
                get_text_im(text='2',nframes=25),
                get_text_im(text='1',nframes=25),
                video_data,
                get_text_im(text=codes[c],nframes=250)),axis=0)
    
    save_video(arr,os.path.join(ofdir,f'gif_{c+1:03d}.gif'))
    
    plt.figure(figsize=(20,10))
    plt.subplot(1,3,1)
    plt.imshow(arr[0,:,:,:])
    plt.axis('off')
    plt.subplot(1,3,2)
    plt.imshow(arr[126,:,:,:])
    plt.axis('off')
    plt.subplot(1,3,3)
    plt.imshow(arr[-1,:,:,:])
    plt.axis('off')
    plt.suptitle(mturk_categories[c],fontsize=20,y=0.66)
    plt.savefig(os.path.join(ofdir,f'im_{c+1:03d}.jpg'))
    plt.close()

100%|███████████████████████████████████████████| 80/80 [03:10<00:00,  2.38s/it]


In [33]:
### BG VIDEOS
indir_root = './Data/pytorch-data/xl_121_15_action_data_bg/test/'
vid_fn_temp = './Data/haa500_v1_1_yolo_seg4/{this_cat}/{this_cat}_{vid_num}_item_0_inp_seg.mp4'
ofdir = './Data/01_gifs_for_mTurk_bg/'
safe_mkdir(ofdir)

codes = [f'{np.random.randint(low=0,high=999):03d}' for _ in range(n)]
df['gif_code_bg'] = codes

for c in tqdm(range(n)):
    this_cat = mturk_categories[c]
    indir = os.path.join(indir_root,this_cat)
    frames = [frame for frame in os.listdir(indir) if not frame.startswith('.')]
    frames.sort()
    frame = frames[0]
    vid_num = frame.split('_')[0]
    vid_fn = vid_fn_temp.format(this_cat=this_cat,vid_num=vid_num)
    video_data = load_video(vid_fn)
    video_data = video_data[0:min(video_data.shape[0],150),:,:,:]
    
    arr = np.concatenate((
                get_text_im(text='5',nframes=25),
                get_text_im(text='4',nframes=25),
                get_text_im(text='3',nframes=25),
                get_text_im(text='2',nframes=25),
                get_text_im(text='1',nframes=25),
                video_data,
                get_text_im(text=codes[c],nframes=250)),axis=0)
    
    save_video(arr,os.path.join(ofdir,f'gif_{c+1:03d}.gif'))
    
    plt.figure(figsize=(20,10))
    plt.subplot(1,3,1)
    plt.imshow(arr[0,:,:,:])
    plt.axis('off')
    plt.subplot(1,3,2)
    plt.imshow(arr[126,:,:,:])
    plt.axis('off')
    plt.subplot(1,3,3)
    plt.imshow(arr[-1,:,:,:])
    plt.axis('off')
    plt.suptitle(mturk_categories[c],fontsize=20,y=0.66)
    plt.savefig(os.path.join(ofdir,f'im_{c+1:03d}.jpg'))
    plt.close()

100%|███████████████████████████████████████████| 80/80 [03:11<00:00,  2.40s/it]


In [34]:
### SEG VIDEOS
indir_root = './Data/pytorch-data/xl_121_15_action_data_seg/test/'
vid_fn_temp = './Data/haa500_v1_1_yolo_seg4/{this_cat}/{this_cat}_{vid_num}_item_0_fwd_seg.mp4'
ofdir = './Data/01_gifs_for_mTurk_seg/'
safe_mkdir(ofdir)

codes = [f'{np.random.randint(low=0,high=999):03d}' for _ in range(n)]
df['gif_code_seg'] = codes

for c in tqdm(range(n)):
    this_cat = mturk_categories[c]
    indir = os.path.join(indir_root,this_cat)
    frames = [frame for frame in os.listdir(indir) if not frame.startswith('.')]
    frames.sort()
    frame = frames[0]
    vid_num = frame.split('_')[0]
    vid_fn = vid_fn_temp.format(this_cat=this_cat,vid_num=vid_num)
    video_data = load_video(vid_fn)
    video_data = video_data[0:min(video_data.shape[0],150),:,:,:]
    
    arr = np.concatenate((
                get_text_im(text='5',nframes=25),
                get_text_im(text='4',nframes=25),
                get_text_im(text='3',nframes=25),
                get_text_im(text='2',nframes=25),
                get_text_im(text='1',nframes=25),
                video_data,
                get_text_im(text=codes[c],nframes=250)),axis=0)
    
    save_video(arr,os.path.join(ofdir,f'gif_{c+1:03d}.gif'))
    
    plt.figure(figsize=(20,10))
    plt.subplot(1,3,1)
    plt.imshow(arr[0,:,:,:])
    plt.axis('off')
    plt.subplot(1,3,2)
    plt.imshow(arr[126,:,:,:])
    plt.axis('off')
    plt.subplot(1,3,3)
    plt.imshow(arr[-1,:,:,:])
    plt.axis('off')
    plt.suptitle(mturk_categories[c],fontsize=20,y=0.66)
    plt.savefig(os.path.join(ofdir,f'im_{c+1:03d}.jpg'))
    plt.close()

100%|███████████████████████████████████████████| 80/80 [02:58<00:00,  2.23s/it]


In [58]:
### VALIDATION VIDEOS
indir_root = './Data/pytorch-data/xl_121_15_action_data_orig/val/'
vid_fn_temp = './Data/haa500_v1_1_yolo_seg4/{this_cat}/{this_cat}_{vid_num}.mp4'
ofdir = './Data/01_gifs_for_mTurk_val/'
safe_mkdir(ofdir)

#codes = [f'{np.random.randint(low=0,high=999):03d}' for _ in range(n)]
#df['gif_code_orig'] = codes

for c in tqdm(range(n)):
    this_cat = mturk_categories[c]
    indir = os.path.join(indir_root,this_cat)
    frames = [frame for frame in os.listdir(indir) if not frame.startswith('.')]
    frames.sort()
    frame = frames[0]
    vid_num = frame.split('_')[0]
    vid_fn = vid_fn_temp.format(this_cat=this_cat,vid_num=vid_num)
    video_data = load_video(vid_fn)
    video_data = video_data[0:min(video_data.shape[0],100),:,:,:]
    
#     arr = np.concatenate((
#                 get_text_im(text='5',nframes=25),
#                 get_text_im(text='4',nframes=25),
#                 get_text_im(text='3',nframes=25),
#                 get_text_im(text='2',nframes=25),
#                 get_text_im(text='1',nframes=25),
#                 video_data,
#                 get_text_im(text=codes[c],nframes=250)),axis=0)
    
    save_video(video_data,os.path.join(ofdir,f'vgif_{c+1:03d}.gif'))
    #save_video(arr,os.path.join(ofdir,f'gif_{c+1:03d}.gif'))
    
#     plt.figure(figsize=(20,10))
#     plt.subplot(1,3,1)
#     plt.imshow(arr[0,:,:,:])
#     plt.axis('off')
#     plt.subplot(1,3,2)
#     plt.imshow(arr[126,:,:,:])
#     plt.axis('off')
#     plt.subplot(1,3,3)
#     plt.imshow(arr[-1,:,:,:])
#     plt.axis('off')
#     plt.suptitle(mturk_categories[c],fontsize=20,y=0.66)
#     plt.savefig(os.path.join(ofdir,f'im_{c+1:03d}.jpg'))
#     plt.close()

100%|███████████████████████████████████████████| 80/80 [01:01<00:00,  1.29it/s]


In [49]:
df = df[['action', 'cat_idx', 'file', 'label',
       'choice_idx_1', 'choice_idx_2', 'choice_idx_3', 'choice_idx_4',
       'choice_idx_5', 'choice_label_1', 'choice_label_2', 'choice_label_3',
       'choice_label_4', 'choice_label_5', 'gif_code_orig', 'gif_code_bg',
       'gif_code_seg']]

In [53]:
df['file'] = [f'gif_{c+1:03d}.gif' for c in range(n)]

/tmp/ipykernel_42927/1520840460.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['file'] = [f'gif_{c+1:03d}.gif' for c in range(n)]


In [55]:
#df.to_csv('./Results/mTurk_masterfile_2.csv')

In [59]:
#!zip -r ./Data/01_gifs_for_mTurk_orig.zip ./Data/01_gifs_for_mTurk_orig

In [61]:
#!zip -r ./Data/01_gifs_for_mTurk_bg.zip ./Data/01_gifs_for_mTurk_bg

In [63]:
#!zip -r ./Data/01_gifs_for_mTurk_seg.zip ./Data/01_gifs_for_mTurk_seg

In [60]:
#!zip -r ./Data/01_gifs_for_mTurk_val.zip ./Data/01_gifs_for_mTurk_val

In [11]:
pwd

'/mmfs1/data/aglinska/BC-actionpred-seg'

In [55]:
plt.ioff()

In [73]:
durs = []
for i in tqdm( np.arange(1,80+1)):
    video_data_orig = load_video(f'./Data/01_gifs_for_mTurk_orig/gif_{i:03d}.gif')
    durs.append( (video_data_orig.shape[0]-125-250)/25 )

100%|███████████████████████████████████████████| 80/80 [00:39<00:00,  2.01it/s]


In [74]:
durs = np.array(durs)
(durs.min(),durs.max())

(0.4, 6.0)

In [3]:
# for i in tqdm( np.arange(1,80+1) ):
    
#     video_data_orig = load_video(f'./Data/01_gifs_for_mTurk_orig/gif_{i:03d}.gif')
#     video_data_bg = load_video(f'./Data/01_gifs_for_mTurk_bg/gif_{i:03d}.gif')
#     video_data_seg = load_video(f'./Data/01_gifs_for_mTurk_seg/gif_{i:03d}.gif')
#     video_data_val = load_video(f'./Data/01_gifs_for_mTurk_val/vgif_{i:03d}.gif')

#     mid1 = 125+int((video_data_orig.shape[0]-125-250)/2)
#     mid2 = int(video_data_val.shape[0]/2)

#     plt.figure(figsize=np.array((15,5))*1.5)
#     plt.subplot(1,4,1)
#     plt.imshow(video_data_orig[mid1,:,:,:])
#     plt.xticks([]);plt.yticks([]);
#     plt.ylabel(df['label'].values[i-1],fontsize=16,fontweight='bold')

#     plt.subplot(1,4,2)
#     plt.imshow(video_data_seg[mid1,:,:,:])
#     plt.xticks([]);plt.yticks([]);
#     #plt.ylabel('golf',fontsize=14)

#     plt.subplot(1,4,3)
#     plt.imshow(video_data_bg[mid1,:,:,:])
#     plt.xticks([]);plt.yticks([]);
#     #plt.ylabel('golf',fontsize=14)

#     plt.subplot(1,4,4)
#     plt.imshow(video_data_val[mid2,:,:,:])
#     plt.xticks([]);plt.yticks([]);
#     #plt.ylabel('golf',fontsize=14)

#     plt.tight_layout()

#     plt.savefig(f'./Data/01-figs-for-ethics/{i:03d}.png',transparent=True)
#     plt.close()

In [2]:
# i = 51
# video_data_orig = load_video(f'./Data/01_gifs_for_mTurk_orig/gif_{i:03d}.gif')
# print(video_data_orig.shape)

# video_data_orig = video_data_orig[125:video_data_orig.shape[0]-250,:,:,:]
# plt.imshow(video_data_orig[-1,:,:,:])
# plt.show()

In [1]:
# for f in np.linspace(0,53,6).astype(int):
#     plt.imshow(video_data_orig[f,:,:,:])
#     plt.axis('off')
#     plt.show()

In [67]:
#i = 1
for i in tqdm( np.arange(1,80+1) ):
    video_data_val = load_video(f'./Data/01_gifs_for_mTurk_val/vgif_{i:03d}.gif')
    i_frames = np.linspace(start=0,stop=video_data_val.shape[0]-1,num=5).astype(int)

    plt.figure(figsize=(20,10))
    c = 0
    for f in i_frames:
        c = c+1
        plt.subplot(1,5,c)
        plt.imshow(video_data_val[f,:,:,:])
        plt.axis('off')
    lbl = df['label'].values[i-1]
    plt.suptitle(lbl,y=.65,fontsize=25)
    plt.tight_layout()


    plt.savefig(os.path.join('./Data/01-still-mturk-train/',lbl.replace(' ','_')+'.jpg'),bbox_inches='tight')
    #plt.show()

 25%|██▌       | 20/80 [00:08<00:26,  2.24it/s]<ipython-input-67-96e9dedfe661>:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(20,10))
100%|██████████| 80/80 [00:36<00:00,  2.19it/s]


In [69]:
#!zip -r './Data/01-still-mturk-train.zip' './Data/01-still-mturk-train/'

In [ ]:
for i in tqdm( np.arange(1,80+1) ):
    
    #video_data_orig = load_video(f'./Data/01_gifs_for_mTurk_orig/gif_{i:03d}.gif')
    #video_data_bg = load_video(f'./Data/01_gifs_for_mTurk_bg/gif_{i:03d}.gif')
    #video_data_seg = load_video(f'./Data/01_gifs_for_mTurk_seg/gif_{i:03d}.gif')
    #video_data_val = load_video(f'./Data/01_gifs_for_mTurk_val/vgif_{i:03d}.gif')

    mid1 = 125+int((video_data_orig.shape[0]-125-250)/2)
    mid2 = int(video_data_val.shape[0]/2)

    plt.figure(figsize=np.array((15,5))*1.5)
    plt.subplot(1,4,1)
    plt.imshow(video_data_orig[mid1,:,:,:])
    plt.xticks([]);plt.yticks([]);
    plt.ylabel(df['label'].values[i-1],fontsize=16,fontweight='bold')

    plt.subplot(1,4,2)
    plt.imshow(video_data_seg[mid1,:,:,:])
    plt.xticks([]);plt.yticks([]);
    #plt.ylabel('golf',fontsize=14)

    plt.subplot(1,4,3)
    plt.imshow(video_data_bg[mid1,:,:,:])
    plt.xticks([]);plt.yticks([]);
    #plt.ylabel('golf',fontsize=14)

    plt.subplot(1,4,4)
    plt.imshow(video_data_val[mid2,:,:,:])
    plt.xticks([]);plt.yticks([]);
    #plt.ylabel('golf',fontsize=14)

    plt.tight_layout()

    plt.savefig(f'./Data/01-figs-for-ethics/{i:03d}.png',transparent=True)
    plt.close()

In [33]:
# for f in tqdm(range(video_data_orig.shape[0])):
#     plt.imshow(video_data_orig[f,:,:,:])
#     plt.axis('off')
#     plt.savefig(f'./Data/xxx-temp-frames/{f:04d}.jpg',transparent=True)

In [19]:
#!zip -r './Data/xxx-temp-frames.zip' './Data/xxx-temp-frames/'

'./Data/xxx-temp-frames/0051.jpg'

In [16]:
#!zip -r './Data/01-figs-for-ethics.zip' './Data/01-figs-for-ethics/'